### Implement Naive Bayes to predict tweet sentiments

In [140]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import twitter_samples

In [141]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/rishushrivastava/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [142]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rishushrivastava/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load and Analyse your data

In [143]:
# load positive tweets
positive_tweets = twitter_samples.strings('positive_tweets.json')
positive_tweets[:3]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!']

In [144]:
# load negative tweets
negative_tweets = twitter_samples.strings('negative_tweets.json')
negative_tweets[:3]

['hopeless for tmr :(',
 "Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(",
 '@Hegelbon That heart sliding into the waste basket. :(']

In [145]:
## total number of pos and neg tweets

print(f"Total No. of Positive tweets: {len(positive_tweets)}")
print(f'Total No. of Negative tweets: {len(negative_tweets)}')

Total No. of Positive tweets: 5000
Total No. of Negative tweets: 5000


In [146]:
## generate a train and test dataset with equal combination of pos and neg tweets
## in total 1000 words, dividing the list of tweets into 8000 train and 2000 test datasets.

train_pos = positive_tweets[:4000]
train_neg = negative_tweets[:4000]

test_pos = positive_tweets[4000:]
test_neg = negative_tweets[4000:]

# combining all of them together

train_data = train_pos + train_neg
test_data = test_pos + test_neg

print(f'Total number of data count train data: {len(train_data)} and test data : {len(test_data)}')

Total number of data count train data: 8000 and test data : 2000


In [223]:
# creating labels for the datasets
train_label = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_label = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

print(f'Shape of Train and Test labels : {train_label.shape} and {test_label.shape}')

Shape of Train and Test labels : (8000,) and (2000,)


### Processing of the words to create word frequencies

In [224]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re

def clean_tweet(tweet):
    '''
        clean the tweet to tokenise, remove stop words and stem the words
    '''
    stop_words = stopwords.words('english')
    #print(f'Total stop words in the vocab: {len(stop_words)}')
    
    stemmer = PorterStemmer()
    
    tweet = re.sub(r'#','',tweet) ## remove the # symbol
    tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet) ## remove any hyperlinks
    tweet = re.sub(r'^RT[\s]+','',tweet) ## remove any Retweets (RT)
    
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_token = tokenizer.tokenize(tweet)
    
    tweet_cleaned = []
    
    for word in tweet_token:
        if word not in stop_words:
            stemmed_word = stemmer.stem(word)
            tweet_cleaned.append(stemmed_word)
            
    return tweet_cleaned
    

def build_tweet_frequency(tweets, label):
    '''
        Build a vocab of tweet word frequencies across corpus. 
        @input: Tweets - list of tweets
                label - Array of tweet sentiments
        @output: a dict of (word, label):frequency
    '''
    label_list = np.squeeze(label).tolist()
    
    freq = {}
    
    for t, l in zip(tweets, label_list):
        for word in clean_tweet(t):
            word_pair = (word,l)
            
            if word_pair in freq:
                freq[word_pair] += 1
            else:
                freq[word_pair] = 1

    return freq

In [225]:
tweet_freq_vocab = build_tweet_frequency(train_data, train_label)

In [226]:
tweet_freq_vocab.get(('happi',1))

161

In [227]:
# testing some tweets
tweets = ['i am happy', 'i am tricked', 'i am sad', 'i am tired', 'i am tired']
tweets_y = [1,0,0,0,0]

build_tweet_frequency(tweets, tweets_y)

{('happi', 1): 1, ('trick', 0): 1, ('sad', 0): 1, ('tire', 0): 2}

### Implementing Naive Bayes Model

![bayes1](images/bayes1.png)

![naive bayes2](images/naivebayes2.png)

![logprior and loglikelihood](images/logprior_and_loglikelhood.png)

In [228]:
def lookup(freqs, word, label):
    '''
        Lookup to find word
    '''
    n = 0
    pair = (word, label)
    
    if (pair in freqs):
        n = freqs[pair]
        
    return n

In [229]:
def train_naive_bayes_model(x,y,freqs):
    '''
        @input:
            - x: input a list of train data
            - y: input a list of labels
            - freqs: the frequency vocab
        @output:
            - loglikelihood: 
            - logprior
    '''
    
    logprior = 0
    loglikelihood = {}
    
    #total number of distinct words in the vocab. Remember words can be in both neg and pos
    vocab = set([items[0] for items in freqs])
    V = len(vocab) 
    
    N_pos = N_neg = V_pos = V_neg = 0
    for pair in freqs.keys():
        
        if pair[1] > 0:
            V_pos += 1 # increment the count of unique positive words by 1
            N_pos += freqs[pair] # Increment the number of positive words by the count for this (word, label) pair
            
        else:    
            V_neg += 1 # increment the count of unique negative words by 1
            N_neg += freqs[pair] # increment the number of negative words by the count for this (word,label) pair
    
    # the number of documents
    D = len(y)
    
    D_pos = np.where(train_label > 0)[0].size
    D_neg = np.where(train_label <= 0)[0].size
    
    logprior = np.log(D_pos) - np.log(D_neg) ## calculate log prior value
    
    
    for word in vocab: ##in vocab
        
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)
        
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
        
    return logprior, loglikelihood
    

In [230]:
logprior, loglikelihood = train_naive_bayes_model(train_data, train_label,tweet_freq_vocab)
print(logprior)
print(len(loglikelihood))

0.0
9120


In [231]:
loglikelihood

{'elizabetholsen': 0.6789369162913185,
 'abe': 0.6789369162913185,
 'brilliantli': 0.6789369162913185,
 'winter': -0.7073574448285721,
 'flesh': -0.014210264268626845,
 'medit': 0.6789369162913185,
 'teen': -0.41967537237679137,
 'play': -0.06550355865617752,
 'lloyd': -0.7073574448285721,
 'mixer': -0.7073574448285721,
 '19:30': 0.6789369162913185,
 'yulin': -0.7073574448285721,
 'hyungbot': 0.6789369162913185,
 '💘': 0.273471808183154,
 'berigaud': -0.7073574448285721,
 'wrist': -0.7073574448285721,
 'sibuk': 0.6789369162913185,
 'pro': 1.5952276481654735,
 'monster': 0.6789369162913185,
 'nikeindonesia': 0.6789369162913185,
 'awar': 0.6789369162913185,
 'soshi': -0.7073574448285721,
 'rito': -0.7073574448285721,
 'basara': -0.7073574448285721,
 'hushedcallwithfraydo': 0.6789369162913185,
 'anyon': -0.6428189236910011,
 'fmt': 0.6789369162913185,
 '0345': 1.0844020243994827,
 'olur': 0.6789369162913185,
 'budg': -0.7073574448285721,
 'brave': 0.3912548438395375,
 'braid': -1.112822552

# Predicting

In [232]:
def predict_nb(tweet, logprior, loglikelihood):
    '''
        Prdict the Naive Bayes
    '''
    word_l = clean_tweet(tweet)
    
    p = 0
    p += logprior
    
    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
            
    if p > 0:
        sentiment = 'Positive'
    else:
        sentiment = 'Negative'
        
    #print(f'Tweet: {tweet} \t Sentiment: {p}')
    return p, sentiment

In [233]:
mytweet = 'i am happy'

predict_value, sentiment = predict_nb(mytweet, logprior, loglikelihood)

print(f'Tweet         : {mytweet}')
print(f'Predict value : {predict_value}')
print(f'Sentiment     : {sentiment}')

Tweet         : i am happy
Predict value : 2.1289470917973166
Sentiment     : Positive


In [234]:
mytweet = 'i am sad'

predict_value, sentiment = predict_nb(mytweet, logprior, loglikelihood)

print(f'Tweet         : {mytweet}')
print(f'Predict value : {predict_value}')
print(f'Sentiment     : {sentiment}')

Tweet         : i am sad
Predict value : -2.8375713118818315
Sentiment     : Negative


In [235]:
mytweet = 'she smiled and was also angry with me. But its OK'

predict_value, sentiment = predict_nb(mytweet, logprior, loglikelihood)

print(f'Tweet          : {mytweet}')
print(f'Predict value  : {predict_value}')
print(f'Sentiment      : {sentiment}')

Tweet          : she smiled and was also angry with me. But its OK
Predict value  : 1.824055465233894
Sentiment      : Positive


# Accuracy

In [236]:
def test_nb(test_x, test_y, logprior, loglikelihood):
    '''
        Testing the naive bayes algorithm
    '''
    
    accuracy = 0
    
    predict_labels = []
    
    for tweets in test_x:
        
        predict_y, sentiment = predict_nb(tweets, logprior, loglikelihood)
        
        if predict_y > 0:
            p_labels = 1
        else:
            p_labels = 0
            
        predict_labels.append(p_labels)
        
    error = np.mean(np.absolute(test_y-predict_labels))
    
    accuracy = 1 - error
    
    return error, accuracy

In [237]:
error, acc = test_nb(test_data, test_label, logprior, loglikelihood)

print(f'Error    : {error}')
print(f'Accuracy : {acc*100}%')

Error    : 0.004
Accuracy : 99.6%
